<a href="https://colab.research.google.com/github/Ry02024/BookManager/blob/main/BookManager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##環境構築

In [22]:
!pip install streamlit streamlit-webrtc pyzbar pillow requests opencv-python-headless pyngrok -q

In [ ]:
!sudo apt-get install libzbar0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libzbar0 is already the newest version (0.23.92-4build2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
import streamlit as st
from pyngrok import ngrok

In [ ]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [ ]:
from pyngrok import ngrok
import os

# ngrokトークンを設定
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

##実行ファイル

###version1

In [26]:
%%writefile streamlit_app.py
import streamlit as st
from streamlit_webrtc import webrtc_streamer, VideoTransformerBase
import cv2
import numpy as np
from pyzbar.pyzbar import decode
import requests
from PIL import Image
from io import BytesIO

# APIから書籍データを取得する関数
def get_book_info(isbn):
    url = f'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "items" in data:
            return data["items"][0]["volumeInfo"]
    return None

# 手動でISBN番号を入力する関数
def manual_isbn_input():
    isbn = st.text_input("ISBN番号を入力してください（例: 9784297108434）")
    if isbn:
        display_book_info(isbn)

# カメラでバーコードを読み取るクラス
class BarcodeScanner(VideoTransformerBase):
    def __init__(self):
        self.last_barcode = None

    def transform(self, frame):
        img = frame.to_ndarray(format="bgr24")
        decoded_objects = decode(img)

        for obj in decoded_objects:
            barcode_data = obj.data.decode("utf-8")
            barcode_type = obj.type

            # デバッグ: 認識したバーコード情報を表示
            st.write(f"認識したバーコード: {barcode_data} (タイプ: {barcode_type})")

            # ISBNは通常 "97" で始まる
            if barcode_type in ["EAN13", "ISBN13"] and barcode_data.startswith('97') and barcode_data != self.last_barcode:
                self.last_barcode = barcode_data
                st.session_state['barcode'] = barcode_data
                break

        return img

# カメラでバーコードを読み取る関数
def camera_isbn_input():
    if 'barcode' not in st.session_state:
        st.session_state['barcode'] = None
    webrtc_ctx = webrtc_streamer(key="barcode-scanner", video_transformer_factory=BarcodeScanner)
    if st.session_state['barcode']:
        barcode = st.session_state['barcode']
        st.success(f"バーコードが読み取られました: {barcode}")
        display_book_info(barcode)
    else:
        st.write("カメラをバーコードに向けてください。")

# 画像からバーコードを読み取る関数
def image_isbn_input():
    uploaded_file = st.file_uploader("バーコードを含む画像をアップロードしてください", type=["png", "jpg", "jpeg"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption='アップロードされた画像', use_column_width=True)
        img_array = np.array(image.convert('RGB'))
        decoded_objects = decode(img_array)
        for obj in decoded_objects:
            barcode_data = obj.data.decode("utf-8")
            barcode_type = obj.type
            if barcode_type in ["EAN13", "ISBN13"] and barcode_data.startswith('97'):
                st.session_state['barcode'] = barcode_data
                break
        if 'barcode' in st.session_state and st.session_state['barcode']:
            st.success(f"バーコードが読み取られました: {st.session_state['barcode']}")
            display_book_info(st.session_state['barcode'])
        else:
            st.error("バーコードが検出されませんでした。")

# 書籍情報を表示する関数
def display_book_info(isbn):
    book_info = get_book_info(isbn)
    if book_info:
        st.write("書籍情報:")
        st.write(f"**タイトル**: {book_info.get('title', '不明')}")
        st.write(f"**著者**: {', '.join(book_info.get('authors', ['不明']))}")
        st.write(f"**出版年**: {book_info.get('publishedDate', '不明')}")
        st.write(f"**出版社**: {book_info.get('publisher', '不明')}")
        st.write(f"**説明**: {book_info.get('description', '不明')}")
        if 'imageLinks' in book_info and 'thumbnail' in book_info['imageLinks']:
            image_url = book_info['imageLinks']['thumbnail']
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))
            st.image(img, caption=book_info.get('title', '不明'))
    else:
        st.error("書籍情報が見つかりませんでした。")

st.title("ISBN番号で書籍データを検索するアプリ")

option = st.radio("選択してください:", ("ISBN番号を入力", "カメラでバーコードを読み取る", "画像からバーコードを読み取る"))

if option == "ISBN番号を入力":
    manual_isbn_input()
elif option == "カメラでバーコードを読み取る":
    camera_isbn_input()
elif option == "画像からバーコードを読み取る":
    image_isbn_input()


Overwriting streamlit_app.py


##version2

##アプリの起動

In [27]:
# ngrokを介してStreamlitを公開
public_url = ngrok.connect(addr='8501')
print('Public URL:', public_url)

Public URL: NgrokTunnel: "https://6fa0-35-197-80-48.ngrok-free.app" -> "http://localhost:8501"


In [28]:
# Streamlitアプリケーションの起動
!streamlit run streamlit_app.py >/dev/null

/content/streamlit_app.py:54: DeprecationWarning: The argument video_transformer_factory is deprecated. Use video_processor_factory instead.
See https://github.com/whitphx/streamlit-webrtc#for-users-since-versions-020
  webrtc_ctx = webrtc_streamer(key="barcode-scanner", video_transformer_factory=BarcodeScanner)
/content/streamlit_app.py:54: DeprecationWarning: The argument video_transformer_factory is deprecated. Use video_processor_factory instead.
See https://github.com/whitphx/streamlit-webrtc#for-users-since-versions-020
  webrtc_ctx = webrtc_streamer(key="barcode-scanner", video_transformer_factory=BarcodeScanner)


In [ ]:

import requests

# APIから書籍データを取得する関数
def get_book_info(isbn):
    url = f'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "items" in data:
            return data["items"][0]["volumeInfo"]
    return None

# テストするISBN番号
isbn = '9784297108434'
book_info = get_book_info(isbn)

if book_info:
    print("書籍情報:")
    print(f"タイトル: {book_info.get('title', '不明')}")
    print(f"著者: {', '.join(book_info.get('authors', ['不明']))}")
    print(f"出版年: {book_info.get('publishedDate', '不明')}")
    print(f"出版社: {book_info.get('publisher', '不明')}")
    print(f"説明: {book_info.get('description', '不明')}")
    if 'imageLinks' in book_info and 'thumbnail' in book_info['imageLinks']:
        print(f"カバー画像URL: {book_info['imageLinks']['thumbnail']}")
else:
    print("書籍情報が見つかりませんでした。")

書籍情報:
タイトル: Kaggleで勝つデータ分析の技術
著者: 門脇大輔, 阪田隆司, 保坂桂佑, 平松雄司
出版年: 2019-10
出版社: 不明
説明: 不明
カバー画像URL: http://books.google.com/books/content?id=SY-KygEACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api
